In [ ]:
# Import the necessary libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.preprocessing import image
from keras.applications import vgg16

In [ ]:
# How to use a pretrained Neural Network to object recognition

import numpy as np
from keras.preprocessing import image
from keras.applications import vgg16

# Load keras VGG16 model that ws prtrained using the imagenet dataset
model = vgg16.VGG16()

# Load the image file, resizing it to (224, 224) pixel (required by this model)
img = image.image_utils.load_img('bay.jpg', target_size=(224,224))

# Convert it to a numpy array
x = image.image_utils.img_to_array(img)

# Add a fourth dimension(since keras expects a list of images)
x = np.expand_dims(x, axis=0)

# Normalize the input_image pixel value range between 0 and 1
x = vgg16.preprocess_input(x)

# Run the image through the deep neural network to make a prediction
predictions = model.predict(x)

# look up the names of the predicted classes
predicted_classes = vgg16.decode_predictions(predictions, top=9)

print('Top predictions for this image:')

for imagenet_id, name, likelihood in predicted_classes[0]:
    print('Prediction: {} - {:2f}'.format(name, likelihood))

In [ ]:
from pathlib import Path


dog = Path('dogs')
non_dog = Path('non_dogs')

In [ ]:
images=[]
labels =[]

In [ ]:
# Load the dog and non dog images to images and their labels to labels
for img in dog.glob('*.png'):
    img = image.image_utils.load_img(img)
    image_array = image.image_utils.img_to_array(img)
    images.append(image_array)
    labels.append(1)
for img in non_dog.glob('*.png'):
    img = image.image_utils.load_img(img)
    image_array = image.image_utils.img_to_array(img)
    images.append(image_array)
    labels.append(0)

In [ ]:
# Convert the images and labels into a numpy array
x_train = np.array(images)
y_train = np.array(labels)

In [ ]:
# Normalize image data to 0-1 range
x_train = vgg16.preprocess_input(x_train)

# Load a pretrained NN 
pretrained_nn = vgg16.VGG16(weight='imagenet',
                           include_top=False,
                           input_shape=(64, 64, 3))

# Extract features for each image (all in one pass)
feature_x = pretrained_nn.predict(x_train)

# Save the array of the extracted features to a file
joblib.dump(features_x, 'x_train.dat')

# Save the matching array of expected values to a file
joblib.dump(y_train, 'y_train.dat')

In [ ]:
# Training with extracted features

#load data set
x_train = joblib.load('x_train.dat')
y_train = joblib.load('y_train.dat')

# Create a model and add layers
model = Sequential()

model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


# Compile the model
model.compile(
            loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

# Train the model
model.fit(x_train,
         y_train,
         epochs=10,
         shuffle=True
         )
# Save neural network structure
model_structure = model.to_json()
f = path('model_structure.json')
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights('model_weights.h5')

In [ ]:
# Making predictions with transfer learning
from keras.models import model_from_json


# Loading the json file that contain the model's structure
f = Path('model_structure.json')
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weight
model.load_weight('model_weights.h5')

#Load an image file to test, resizing it to 64 * 64 
img = image.image_utils.load_img('dog.png', target_size=(64, 64))

# Convert the image to a numpy array
image_array = image.image_utils.img_to_array(img)

# Add a fourth dimension
images = np.expand_dim(image_array, axis=0)

# Normalize the data
images = vgg16.preprocess_input(images)

# Use the pretrained model to extract features from the test image
feature_extraction_model = vgg16.VGG16(weights='imagenet',
                                      include_top=False,
                                      input_shape=(64,64,3))
features = feature_extraction_model.predict(images)

# Make a final prediction using our own model
results = model.predict(features)

